In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38'
) 

Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/3878086/
Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/3878086/


23/07/06 12:43:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0454:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230706-1242-0.2.113-cf32652c5077.log


In [ ]:
from bokeh.io import show, output_notebook, export_png

from bokeh.layouts import gridplot
output_notebook()
import numpy as np

import bokeh.palettes
from bokeh.models import CategoricalColorMapper

In [4]:
gwas = hl.import_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/external-data/TS_Oct2018', delimiter=' ')
gwas = gwas.annotate(locus = hl.locus(gwas.CHR, hl.int32(gwas.BP), reference_genome='GRCh37'))

rg37 = hl.get_reference('GRCh37')  
rg38 = hl.get_reference('GRCh38')  

2023-07-05 11:37:17.255 Hail: INFO: Reading table without type imputation
  Loading field 'SNP' as type str (not specified)
  Loading field 'CHR' as type str (not specified)
  Loading field 'BP' as type str (not specified)
  Loading field 'A1' as type str (not specified)
  Loading field 'A2' as type str (not specified)
  Loading field 'INFO' as type str (not specified)
  Loading field 'OR' as type str (not specified)
  Loading field 'SE' as type str (not specified)
  Loading field 'P' as type str (not specified)


In [5]:
rg37.add_liftover(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/external-data/references_grch37_to_grch38.over.chain.gz',
    rg38
)

In [7]:
gwas = gwas.annotate(hg39locus = hl.liftover(gwas.locus, 'GRCh38'))

In [ ]:
gtf = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/resources/genecode_43/gencode.v43.basic.annotation.gtf',
    delimiter = "\t",
    no_header = True,
    comment = "#"
)

# filter the gtf to only contain protein-coding genes with any GO term
gtf = gtf.filter((gtf.f2 == 'gene') & (gtf.f8.contains('protein_coding')))

gtf = gtf.select(
    gene_position = hl.locus_interval(
        gtf.f0,
        hl.int(gtf.f3),
        hl.int(gtf.f4),
        reference_genome='GRCh38'
    ),
    gene_symbol = gtf.f8.split("\"")[5]
)  

go_genes = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/resources/human-genes-with-GO-and-symbols'
)

go_genes = go_genes.key_by(go_genes['UniProtKB Gene Name symbol'])

gtf = gtf.key_by(gtf['gene_symbol'])
gtf = gtf.filter(
    hl.is_defined(
        go_genes[gtf.gene_symbol]
    )
)

# overall we are keeping 18303 genes. At this poin the within gene intervals are extended by 20kb each side
start = hl.if_else(
    gtf.gene_position.start.position <= 20000, 1, gtf.gene_position.start.position - 20000
)

contig_len = hl.contig_length(gtf.gene_position.start.contig, reference_genome='GRCh38') 

stop = hl.if_else(
    (contig_len - gtf.gene_position.end.position) <= 20000,
    contig_len,
    gtf.gene_position.end.position + 20000
)

gtf = gtf.annotate(
    interval_20kb = hl.locus_interval(
        gtf.gene_position.start.contig,
        start,
        stop,
        reference_genome='GRCh38'
    )
)

gtf = gtf.key_by(gtf.interval_20kb)

In [8]:
gwas = gwas.annotate(
    nearest_genes_20kb = hl.array(
        hl.set(
            gtf.index(gwas.hg39locus, all_matches=True)['gene_symbol']
        )
    )
)

In [9]:
gwas.checkpoint(localfs_path+'gwas-ts.ht')

2023-07-03 09:27:31.451 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-07-03 09:27:33.764 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-07-03 09:27:34.291 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-07-03 09:27:36.787 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-07-03 09:28:28.648 Hail: INFO: wrote table with 8265318 rows in 3 partitions to /localfs/3878086/gwas-ts.ht
    Total size: 385.89 MiB
    * Rows: 385.89 MiB
    * Globals: 11.00 B
    * Smallest partition: 2293984 rows (106.97 MiB)
    * Largest partition:  3489970 rows (163.69 MiB)


In [4]:
gwas = hl.read_table(localfs_path+'gwas-ts.ht')

#### divide per gene - and calculate which genes are above background (check if any of these genes are from this gwas)

In [5]:
all_genes = gwas.nearest_genes_20kb.collect()
all_genes = [item for sublist in all_genes for item in sublist]
all_genes = list(set(all_genes))

gts_genes = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5', 'PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2',
         'RICTOR', 'WWC1', 'CELSR3, NIPBL', 'FN1', 'PNKD', 'CDH26', 'CADM2', 'OPCML', 'CDH9',
         'NCAM2', 'CD47', 'CDH5', 'CADM4', 'C1QBP', 'CTTN', 'LSAMP',
         'PKP4', 'PCDH1', 'CNTNAP2', 'MBP', 'GABBR2', 'GABBR2', 'GRIK4', 'NCR1', 'FLT3', 'IL12A', 'HDAC9',
         'CD180', 'CDH26', 'NCAM2', 'NTM', 'ROBO2', 'NR2F1', 'ASH1L', 'FLT3']

all_genes = [g for g in all_genes if g not in gts_genes]

In [6]:
np.random.seed(1)
top_genes = ['CHADL', 'DRD2', 'MAOA', 'PCDH10', 'HTR2A', 'SLITRK5', 'SORCS3', 'KCNQ5', 'CDH9', 'CHD8']
colors = bokeh.palettes.plasma(200)

In [7]:
# figure out a way to normalize per gene

In [ ]:
plots = []
from bokeh.plotting import figure, show, output_notebook
output_notebook()

for gene in top_genes:
    
    gwas_top = gwas.filter(hl.any(gwas.nearest_genes_20kb.contains(gene)))
    p_top = gwas_top.P.collect()
    p_hist = gwas_top.aggregate(hl.expr.aggregators.hist(hl.float64(gwas_top.P), 0, 1, 100))
    
    p_values = []
    
    for p in range(100):
        randoms = np.random.randint(len(all_genes), size=1)
        genes = [j for i, j in enumerate(all_genes) if i in randoms]

        p_value = gwas.filter(hl.any(lambda x: hl.literal(genes).contains(x), gwas.nearest_genes_20kb)).P.collect()

        p_values.append(p_value)

    y = [float(i) for i in p_top]
    y = -np.log10(y)
    
    hist_top, edge_top = np.histogram(y, bins = 100)
    
    histograms = []
    edges = []

    for p in p_values:
        x = [float(i) for i in p]
        x = -np.log10(x)
        hist,edge = np.histogram(x, bins = 50)
        histograms.append(hist)
        edges.append(edge)

    p1 = figure(plot_width=800, plot_height=800, title = gene)

    for x in range(100):
        p1.quad(
            top=np.log10(histograms[x]),
            bottom=0,
            left=edges[x][:-1],
            right=edges[x][1:],
            fill_color=colors[100],
            line_color=colors[50],
            alpha=0.05
        )

    p1.quad(top=np.log10(hist_top),bottom=0, left=edge_top[:-1], right=edge_top[1:],
            fill_color=colors[0], line_color=colors[0], alpha=1)

    p1.xaxis.axis_label = '-log10(p_value)'
    p1.yaxis.axis_label = 'log10(number of SNPs)'

    p1.xaxis.axis_label_text_font_size = "15pt"
    p1.xaxis.major_label_text_font_size = "15pt"
    p1.yaxis.axis_label_text_font_size = "15pt"
    p1.yaxis.major_label_text_font_size = "15pt"
    
    show(p1)
    
    plots.append(p1)

Loading BokehJS ...

In [ ]:
# maybe try qq plots + calculate tons of random lambda GCs (any normalisation? confidence intervals?)

In [8]:
p = hl.plot.qq(hl.float(gwas.P))

2023-07-06 12:43:47.003 Hail: INFO: Ordering unsorted dataset with network shuffle
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-07-06 12:44:16.321 Hail: INFO: Ordering unsorted dataset with network shuffle


In [9]:
show(p)

In [19]:
top_genes = ['CHADL','DRD2', 'MAOA', 'PCDH10', 'HTR2A', 'SLITRK5', 'SORCS3', 'KCNQ5', 'CDH9', 'CHD8']

In [20]:
gwas_top = gwas.filter(hl.any(lambda x: hl.literal(top_genes).contains(x), gwas.nearest_genes_20kb))

In [21]:
p2 = hl.plot.qq(hl.float(gwas_top.P))

2023-07-06 12:46:40.222 Hail: INFO: Ordering unsorted dataset with network shuffle
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-07-06 12:46:44.468 Hail: INFO: Ordering unsorted dataset with network shuffle


In [22]:
show(p2)

In [23]:
gwas_top.aggregate(hl.agg.filter(hl.float(gwas_top.P) < 0.00001, hl.agg.counter(gwas_top.nearest_genes_20kb)))

{<FrozenList(frozen=True, ['CHADL', 'L3MBTL2'])>: 8,
 <FrozenList(frozen=True, ['CHADL', 'L3MBTL2', 'RANGAP1'])>: 2}

In [ ]:
#check where these variants are

In [ ]:
lgcs = []

for i in range(1000):
    
    randoms = np.random.randint(len(all_genes), size=10)
    genes = [j for i, j in enumerate(all_genes) if i in randoms]

    gwas_rand = gwas.filter(hl.any(lambda x: hl.literal(genes).contains(x), gwas.nearest_genes_20kb))

    lgcs.append(hl.lambda_gc(hl.float(gwas_rand.P)))

In [41]:
np.percentile(lgcs, 90)

1.5044869974671107

In [44]:
np.percentile(lgcs, 75)

1.2913186320485897

https://stackoverflow.com/questions/74874235/confidence-interval-in-normal-q-q-plot-using-statsmodels